# Capstone Project Part 2

## Code from part 1 to get the data

In [6]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [7]:
def combine_string(a: pd.Series) -> str:
    result: str = a.iloc[0]
    for s in a[1:]:
        result += ','
        result += str(s)
    return result

web = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
bs = BeautifulSoup(web.text, "html.parser")
data_table = bs.find("table")
header = [h.text.strip() for h in data_table.find_all('th')]
data = [[i.text.strip() for i in row.find_all('td')] for row in data_table.find_all('tr')]
df = pd.DataFrame(data[1:])
df.columns = header
df.replace("Not assigned", np.nan, inplace=True)
df = df.dropna()
df = df.groupby(by=['Postal Code', 'Borough']).agg(combine_string).reset_index()

In [8]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Part 2 get coordinates

In [9]:
import geocoder

def get_lat_lng(postal_code: str) -> tuple:
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

Google is unavailable. Use arcgis.

In [10]:
latitude = []
longitude = []
count = 0
for postal_code in df['Postal Code']:
    count += 1
    lat, lng = get_lat_lng(postal_code)
    latitude.append(lat)
    longitude.append(lng)
    print(str(count) + '/' + str(df.shape[0]) + ' ' + postal_code + ' finished.')

1/103 M1B finished.
2/103 M1C finished.
3/103 M1E finished.
4/103 M1G finished.
5/103 M1H finished.
6/103 M1J finished.
7/103 M1K finished.
8/103 M1L finished.
9/103 M1M finished.
10/103 M1N finished.
11/103 M1P finished.
12/103 M1R finished.
13/103 M1S finished.
14/103 M1T finished.
15/103 M1V finished.
16/103 M1W finished.
17/103 M1X finished.
18/103 M2H finished.
19/103 M2J finished.
20/103 M2K finished.
21/103 M2L finished.
22/103 M2M finished.
23/103 M2N finished.
24/103 M2P finished.
25/103 M2R finished.
26/103 M3A finished.
27/103 M3B finished.
28/103 M3C finished.
29/103 M3H finished.
30/103 M3J finished.
31/103 M3K finished.
32/103 M3L finished.
33/103 M3M finished.
34/103 M3N finished.
35/103 M4A finished.
36/103 M4B finished.
37/103 M4C finished.
38/103 M4E finished.
39/103 M4G finished.
40/103 M4H finished.
41/103 M4J finished.
42/103 M4K finished.
43/103 M4L finished.
44/103 M4M finished.
45/103 M4N finished.
46/103 M4P finished.
47/103 M4R finished.
48/103 M4S finished.
4

In [11]:
df.loc[:, 'Latitude'] = latitude
df.loc[:, 'Longitude'] = longitude

In [12]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81153,-79.19552
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17520
3,M1G,Scarborough,Woburn,43.76820,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76969,-79.23944


In [14]:
df.to_csv('~/DataProjects/Coursera_Capstone/output.csv', index=False)